In [ ]:
import io
import os
import sys
import json
import tempfile
import pandas as pd
import numpy as np

from scipy.interpolate import interp1d

import matplotlib.pyplot as plt
%matplotlib inline

from CoolProp.CoolProp import PropsSI

# shortcuts
cwd = os.getcwd()
join = os.path.join
norm = os.path.normpath

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 14

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
def add_delimiters(fpath, delimiter=','):
    
    s_data = ''
    max_num_delimiters = 0
    
    with open(fpath, 'r') as f:
        for line in f:
            s_data += line
            delimiter_count = line.count(delimiter)
            if delimiter_count > max_num_delimiters:
                max_num_delimiters = delimiter_count
    
    s_delimiters = delimiter * max_num_delimiters + '\n'
    
    return io.StringIO(s_delimiters + s_data)

In [ ]:
def load_data(d_files, drop_col=False):
    dfs = []
    
    for key, file in sorted(d_files.items()):
        
        d = pd.read_csv(add_delimiters(file), index_col=0)
                
        if drop_col:
            d.drop(labels='Unnamed: 1', axis=1, inplace=True)
            
        d.index.name = 'time stamp'
            
        d['data'] = key
        dfs.append(d)
    return pd.concat(dfs).reset_index().set_index(['time stamp', 'data'])

In [ ]:
f_dyna = {'Load': 'loads_dynamic.csv',
          'g-val': 'g_dynamic.csv',
          'Width': 'widths_dynamic.csv',
          'Resist': 'widths_dynamic.csv'}

df_dynamic = load_data(f_dyna)
rb = pd.read_csv('resist_dynamic.csv', index_col=0, header=None)

for i in df_dynamic.index.levels[0]:
    df_dynamic.loc[i, 'Resist'] = np.nan
    df_dynamic.loc[i, 'Resist'][0] = rb.loc[i]
    
df_dynamic.loc[120]

In [ ]:
f_stat = {'Load': 'loads_static.csv',
          'g-val': 'g_static.csv',
          'Width': 'widths_static.csv',
          'Resist': 'widths_static.csv'}

df_static = load_data(f_stat)

rb = pd.read_csv('resist_static.csv', index_col=0, header=None)

for i in df_static.index.levels[0]:
    df_static.loc[i, 'Resist'] = np.nan
    df_static.loc[i, 'Resist'][0] = rb.loc[i]
    
df_static.loc[240]

In [ ]:
f_none = {'Load': 'loads_none.csv',
          'g-val': 'g_none.csv',
          'Width': 'widths_none.csv',
          'Resist': 'widths_none.csv'}

df_none = load_data(f_none)

rb = pd.read_csv('resist_none.csv', index_col=0, header=None)

for i in df_none.index.levels[0]:
    df_none.loc[i, 'Resist'] = np.nan
    df_none.loc[i, 'Resist'][0] = rb.loc[i]
    
df_none.loc[240]

In [ ]:
t_g = 16.1
depth = 76.2
soil_cond = 2.7
soil_dens = 2500
soil_cp = 880
c_0 = 1 / (2 * np.pi * soil_cond)
soil_diffusivity = soil_cond / (soil_dens * soil_cp)
ts = depth ** 2 / (9 * soil_diffusivity)

In [ ]:
def calc_history_temp_rise(df, timestep):

    temp_rise_sum = 0

    def get_load(energy, width):
        return energy / width
    
    def calc_temp_rise(energy_i, 
                       energy_i_minus_1, 
                       width_i,
                       width_i_minus_1,
                       g):
        load_i = get_load(energy_i, width_i)
        load_i_minus_1 = get_load(energy_i_minus_1, width_i_minus_1)
        return (load_i - load_i_minus_1) * g * c_0

    num = df.loc[timestep, 'Load'].count()
    
    loads = list(df.loc[timestep, 'Load'].dropna())
    widths = list(df.loc[timestep, 'Width'].dropna())
#     times = list(df.loc[timestep, 'Width'].dropna().cumsum())
    
#     def get_g_wrapped(x):
#         r_b = df.loc[timestep, 'Resist'][0]
#         return get_g_val(x, r_b)
    
#     gs = list(map(get_g_wrapped, times))
    gs = list(df.loc[timestep, 'g-val'].dropna())
       
    for i in range(num - 1):
        energy_i = loads[i]
        energy_i_minus_1 = loads[i + 1]
        width_i = widths[i]
        width_i_minus_1 = widths[i + 1]
        g = gs[i + 1]
        
        temp_rise_sum += calc_temp_rise(energy_i,
                                        energy_i_minus_1,
                                        width_i,
                                        width_i_minus_1,
                                        g)

    energy = loads[-1]
    width = widths[-1]
    g = gs[-1]
    
    temp_rise_sum += get_load(energy, width) * g * c_0
    
    return temp_rise_sum

In [ ]:
calc_history_temp_rise(df_none, 120)

In [ ]:
def calc_MFT(df, timestep):
    energy = df.loc[timestep, 'Load'][0]
    width = df.loc[timestep, 'Width'][0]
    r_b = df.loc[timestep, 'Resist'][0]
    load = energy / width
    
    arr = [t_g,  calc_history_temp_rise(df, timestep), load * r_b]
    arr.append(sum(arr))
    return arr

In [ ]:
calc_MFT(df_none, 120)

In [ ]:
timestamps = list(df_none.index.levels[0])
# timestamps

In [ ]:
def calc_all_MFT(df):
    lst = []
    for time in timestamps:
        lst.append(calc_MFT(df, time))
        
    return lst

In [ ]:
df_d = pd.DataFrame(data=calc_all_MFT(df_dynamic), index=timestamps, columns=['Tg', 'Temp Hist', 'Load Rb', 'MFT'])

In [ ]:
df_s = pd.DataFrame(data=calc_all_MFT(df_static), index=timestamps, columns=['Tg', 'Temp Hist', 'Load Rb', 'MFT'])

In [ ]:
df_n = pd.DataFrame(data=calc_all_MFT(df_none), index=timestamps, columns=['Tg', 'Temp Hist', 'Load Rb', 'MFT'])

In [ ]:
df_exp_raw = pd.read_csv('MFRTRT_loads.csv', parse_dates=True, index_col='Date/Time')
df_exp_raw = df_exp_raw.resample('2T').mean()

In [ ]:
df_exp = df_exp_raw.interpolate(method='linear')
df_exp['Run Time'] = df_exp.index.to_series().diff().dt.total_seconds().cumsum()
df_exp['Run Time'].iloc[0] = 0
df_exp['Run Time'] += 120
df_exp.set_index('Run Time', drop=True, inplace=True)
df_exp['MFT'] = df_exp[['Inlet 1 [C]', 'Outlet 1 [C]']].mean(axis=1)
df_exp.head(2)

In [ ]:
start_pt = 0
end_pt = 8000

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

ax.plot(df_s.loc[start_pt:end_pt]['MFT'], label='Static', linestyle='--')
ax.plot(df_d.loc[start_pt:end_pt]['MFT'], label='Dynamic', linestyle='-.')
ax.plot(df_n.loc[start_pt:end_pt]['MFT'], label='None')
ax.plot(df_exp.loc[start_pt:end_pt]['MFT'], label='Exp', marker='x')

# ax.set_ylim(16, 26)

plt.legend()
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

ax.plot(df_s.loc[start_pt:end_pt]['Temp Hist'], label='Static')
ax.plot(df_d.loc[start_pt:end_pt]['Temp Hist'], label='Dynamic')
ax.plot(df_n.loc[start_pt:end_pt]['Temp Hist'], label='None', linestyle='--')

plt.legend()
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

ax.plot(df_s.loc[start_pt:end_pt]['Load Rb'], label='Static')
ax.plot(df_d.loc[start_pt:end_pt]['Load Rb'], label='Dynamic')
ax.plot(df_n.loc[start_pt:end_pt]['Load Rb'], label='None', linestyle='--')

plt.legend()
plt.show()

In [ ]:
g_data = np.loadtxt('MFRTRT_g_functions.csv', delimiter=',')
g_func = interp1d(g_data[:,0], g_data[:,1], fill_value='extrapolate')

In [ ]:
def get_g_val(t, r_b):
    
    lntts = np.log(t/ts)
    
    g = float(g_func(lntts))
    
    if (g / (2 * np.pi * soil_cond) + r_b) < 0:
        return -r_b * 2 * np.pi * soil_cond
    else:
        return g

In [ ]:
def plot_g_values(df, timestep):
    
    if df == 'none':
        df = df_none
    elif df == 'static':
        df = df_static
    elif df == 'dynamic':
        df = df_dynamic
    
    sim = df.loc[timestep, 'g-val'].values
    widths = df.loc[timestep, 'Width']
    times = widths.cumsum()    

    def get_g_wrapped(x):
        r_b = df.loc[timestep, 'Resist'][0]
        return get_g_val(x, r_b)
    
    act = list(map(get_g_wrapped, times))
    
    lntts = np.log(times / ts)
    
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    
    ax.plot(lntts, act, label='Actual', marker='x')
    ax.plot(lntts, sim, label='Sim', linestyle='--', marker='D')
        
    ax.set_xlim((-16, np.log(timestep/ts)))
    ax.set_ylim((-3, get_g_wrapped(timestep)))
    plt.legend()
    plt.show()
    
    ret_df = pd.DataFrame(data={'times': times, 'lntts': lntts, 'Actual': act, 'Simulation': sim})
    ret_df.set_index('times', inplace=True)
    return ret_df

In [ ]:
df_none.index.levels

In [ ]:
stat_plot_data = plot_g_values('static', 36000)

In [ ]:
dyna_plot_data = plot_g_values('dynamic', 36000)

In [ ]:
none_plot_data = plot_g_values('none', 36000)

In [ ]:
from ipywidgets import interactive
import ipywidgets as widgets

In [ ]:
interactive_plot = interactive(plot_g_values, df='static', timestep=(120, 36000, 120))
output = interactive_plot.children[-1]
output.layout.height = '600px'
interactive_plot

In [ ]:
def plot_delta_g(timestep):
    
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
   
    y = df_static.loc[timestep, 'g-val'].diff().dropna()
    x = df_static.loc[timestep, 'Width'].cumsum().dropna()[1:]
    ax.plot(x, y, label='Static', linestyle='-.', marker='x')
    
    y = df_dynamic.loc[timestep, 'g-val'].diff().dropna()
    x = df_dynamic.loc[timestep, 'Width'].cumsum().dropna()[1:]
    ax.plot(x, y, label='Dynamic', linestyle='--', marker='D')

    y = df_none.loc[timestep, 'g-val'].diff().dropna()
    x = df_none.loc[timestep, 'Width'].cumsum().dropna()[1:]
    ax.plot(x, y, label='None')
    
    ax.set_xlim((0, x[-1]))
    
    plt.legend()
    plt.plot()

In [ ]:
plot_delta_g(10800)